In [ ]:
# Obtain an AAD token with the correct resource ID
import json
import requests

#Setup the endpoint
endpoint = 'https://login.microsoftonline.com/[TENANT_ID]/oauth2/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
payload = 'grant_type=client_credentials&client_id=[SERVICE_PRINCIPAL_CLIENT_ID]&client_secret=[SERVICE_PRINCIPAL_SECRET]&resource=2ff814a6-3304-4ab8-85cb-cd0e6f879c1d'
r = requests.post(endpoint, headers=headers, data=payload)
response = r.json()
#print(response)
bearertoken = response["access_token"]
print(bearertoken)

In [4]:
endpoint = 'https://[DATABRICKS_WORKSPACE_URL]/api/2.0/instance-pools/create'
payload={
  "instance_pool_name": "[POOL_NAME]",
  "node_type_id": "Standard_D3_v2",
  "min_idle_instances": 0, "max_capacity":2
}
headers = {'X-Databricks-Org-Id':'[DATABRICKS_ORG_ID]','Authorization': 'Bearer %s' % bearertoken}
r = requests.post(endpoint, headers=headers, data=json.dumps(payload))
response = r.json()
print(response)

{'instance_pool_id': '0521-145002-meet10-pool-EP6iFpfl'}


In [7]:
instance_pool_id = response['instance_pool_id']
print(instance_pool_id)

0521-145002-meet10-pool-EP6iFpfl


In [59]:
pool_id=response['instance_pool_id']
endpoint = 'https://adb-3919543476320272.12.azuredatabricks.net/api/2.0/instance-pools/delete'
payload={
  "instance_pool_id": pool_id
}
r = requests.post(endpoint, headers=headers, data=json.dumps(payload))
print(r.status_code)

200


In [16]:
#get a auth token for keyvault
#Setup the endpoint
endpoint = 'https://login.microsoftonline.com/244ecd6b-204f-4a0e-b76f-eb3bc132009e/oauth2/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
payload = 'grant_type=client_credentials&client_id=3730f98f-4ff3-4cb0-ad4e-270032067016&client_secret=~iauAuCTblnO6M_O-rG6rcvm4.H0nPFV~H&resource=https%3A%2F%2Fvault.azure.net'
r = requests.post(endpoint, headers=headers, data=payload)
response = r.json()
bearertoken = response["access_token"]
print(bearertoken)

eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkN0VHVoTUptRDVNN0RMZHpEMnYyeDNRS1NSWSIsImtpZCI6IkN0VHVoTUptRDVNN0RMZHpEMnYyeDNRS1NSWSJ9.eyJhdWQiOiJodHRwczovL3ZhdWx0LmF6dXJlLm5ldCIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzI0NGVjZDZiLTIwNGYtNGEwZS1iNzZmLWViM2JjMTMyMDA5ZS8iLCJpYXQiOjE1OTAwNzI3OTcsIm5iZiI6MTU5MDA3Mjc5NywiZXhwIjoxNTkwMDc2Njk3LCJhaW8iOiI0MmRnWURqMCtmSXQxbDJQWjJ0OWVxU3k5T1pXUndBPSIsImFwcGlkIjoiMzczMGY5OGYtNGZmMy00Y2IwLWFkNGUtMjcwMDMyMDY3MDE2IiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvMjQ0ZWNkNmItMjA0Zi00YTBlLWI3NmYtZWIzYmMxMzIwMDllLyIsIm9pZCI6ImE5YjM2YzQwLTcxMTEtNGIxMC05NGE5LTBjZWJmNmJjN2I2OSIsInN1YiI6ImE5YjM2YzQwLTcxMTEtNGIxMC05NGE5LTBjZWJmNmJjN2I2OSIsInRpZCI6IjI0NGVjZDZiLTIwNGYtNGEwZS1iNzZmLWViM2JjMTMyMDA5ZSIsInV0aSI6IlhDQ2NoRXIwLTAyY3NFWXUtSU92QUEiLCJ2ZXIiOiIxLjAifQ.tVy0HVeDXKSyFGp5VLu1jsPzTFOyWZbx7u14Zbg85uxToqerUaByMpUlNgsvvR3nO85S_0oKHyFvUOzntWCmix7Lf89RkRvURX533rpO2mcK-Bh1LzMo36gFBaOpDH3SELM7lKv1c1ZqR-iAgeS3LW2vw1-TznuTWqnEKMFkP27Iz1SOy-WSw02cpa9D-EIT

In [20]:
endpoint = 'https://coocekeyvault.vault.azure.net/secrets/my-adb-pool?api-version=7.0'
payload={
"value": instance_pool_id
}
headers = {'Content-Type': 'application/json','Authorization': 'Bearer %s' % bearertoken}
r = requests.put(endpoint, headers=headers,data=json.dumps(payload))
print(r.text)

{"value":"0521-145002-meet10-pool-EP6iFpfl","id":"https://coocekeyvault.vault.azure.net/secrets/my-adb-pool/a50b36ec13e2489c8807ea55d4a57a7a","attributes":{"enabled":true,"created":1590073674,"updated":1590073674,"recoveryLevel":"Purgeable"}}


This section creates a cluster and submits a job

In [ ]:
endpoint = 'https://adb-3919543476320272.12.azuredatabricks.net/api/2.0/clusters/creae'
headers = {'X-Databricks-Org-Id':'3919543476320272','Authorization': 'Bearer %s' % bearertoken}
r = requests.get(endpoint, headers=headers)
print(r.text)